In [12]:
import random
import os
import cv2
import numpy as np
from pathlib import Path
from PIL import Image
import torch
from tqdm import tqdm
import xml.etree.ElementTree as ET
from ..CoOp.dassl.data.datasets import DATASET_REGISTRY, Datum, DatasetBase

ImportError: attempted relative import with no known parent package

In [3]:
example_prot = torch.load('/home/gridsan/manderson/ovdsat/run/coop_prototypes/boxes/dior_N5-1/prototypes_remoteclip-14.pt')

In [4]:
example_prot

{'prototypes': tensor([[-1.3440e-01,  1.2176e+00,  1.8888e-01,  ...,  4.1843e-01,
           2.5628e-01,  6.7606e-02],
         [-3.8782e-02,  7.0991e-01,  1.5749e-01,  ...,  2.5575e-01,
           3.4312e-01,  3.5771e-01],
         [-2.3660e-01,  2.3818e-02,  1.9806e-01,  ...,  4.8420e-01,
           7.0869e-01, -5.0834e-01],
         ...,
         [ 8.9706e-03,  6.1654e-01,  7.9282e-03,  ...,  5.4082e-01,
           1.0171e-01, -5.0545e-01],
         [-1.3720e-01,  5.8861e-01,  1.6511e-02,  ...,  1.7120e-01,
           2.4579e-01,  4.1367e-01],
         [ 1.9332e-01,  3.0955e-01,  1.0576e-01,  ..., -3.7555e-01,
           6.8911e-04, -2.8699e-01]], requires_grad=True),
 'label_names': ['ground track field',
  'baseball field',
  'bridge',
  'expressway toll station',
  'vehicle',
  'airplane',
  'airport',
  'tennis court',
  'train station',
  'storage tank',
  'stadium',
  'windmill',
  'ship',
  'golf field',
  'overpass',
  'chimney',
  'dam',
  'basketball court',
  'harbor',
  

In [10]:
example_prot['prototypes'].shape

torch.Size([20, 768])

In [5]:
context = torch.load('/home/gridsan/manderson/ovdsat/CoOp/output/dior/CoOp/vit_l14_remote-ep100-ctx2-crop_5shots/nctx4_cscFalse_ctpmiddle/seed1/prompt_learner/model.pth.tar-100', map_location="cpu")

In [8]:
context['state_dict'].keys()

odict_keys(['ctx', 'token_prefix', 'token_suffix'])

In [9]:
print(context['state_dict']['ctx'].shape)
print(context['state_dict']['token_prefix'].shape)
print(context['state_dict']['token_suffix'].shape) # I think the class names might be in here, so order does matter

torch.Size([4, 768])
torch.Size([21, 1, 768])
torch.Size([21, 72, 768])


## Generate crops for few shot training sets
* Run coco_to_seg first to create images with masks (if not done already!)
* Crop images for each N shot and M split and save to separate folder
* Extract 10 random background crops per image and save in separate folder

In [6]:
def apply_rectangular_mask_and_crop(image: Image.Image, mask: Image.Image) -> Image.Image:
    """Crops the image to the rectangular mask bounding box only (no masking)."""
    mask_array = np.array(mask.convert("L"))

    # Get non-zero mask coordinates
    ys, xs = np.where(mask_array > 0)
    if xs.size == 0 or ys.size == 0:
        return Image.new("RGB", (1, 1))  # Blank image if no mask

    x_min, x_max = xs.min(), xs.max()
    y_min, y_max = ys.min(), ys.max()

    # Crop the image directly to the mask's bounding rectangle
    return image.crop((x_min, y_min, x_max + 1, y_max + 1))

def process_directory(input_dir: str, output_dir: str):
    input_path = Path(input_dir)
    output_path = Path(output_dir)

    for root, _, files in os.walk(input_path):
        for file in files:
            if file.endswith(".jpg") and not file.endswith(".mask.jpg"):
                image_file = Path(root) / file
                mask_file = image_file.with_name(f"{image_file.stem}.mask.jpg")

                if not mask_file.exists():
                    print(f"[!] Mask not found for {image_file.name}, skipping.")
                    continue

                try:
                    image = Image.open(image_file)
                    mask = Image.open(mask_file)
                    masked_cropped = apply_rectangular_mask_and_crop(image, mask)

                    # Determine output path
                    relative_path = image_file.relative_to(input_path)
                    output_image_path = output_path / relative_path
                    output_image_path.parent.mkdir(parents=True, exist_ok=True)

                    masked_cropped.save(output_image_path)
                    print(f"Saved: {output_image_path}")
                except Exception as e:
                    print(f"Error processing {image_file}: {e}")

In [48]:
# For few-shot train

for N in [5, 10, 30]:
    for M in [1]:
        print(f'================ Processing train N={N}, M={M} ================')
        input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_N{N}-{M}'
        output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N{N}-{M}'
        process_directory(input_root, output_root)
        print()

================ Processing train N=5, M=1 ================
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N5-1/ship/11728.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N5-1/ship/03398.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N5-1/ship/01572.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N5-1/ship/12329.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N5-1/ship/22752.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N5-1/harbor/00364.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N5-1/harbor/15432.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N5-1/harbor/12862.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N5-1/harbor/08505.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N5-1/Expressway-toll-station/148

In [49]:
# For val

for M in [1]:
    print(f'================ Processing val M={M} ================')
    input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_val-{M}'
    output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-{M}'
    process_directory(input_root, output_root)
    print()

================ Processing val M=1 ================
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-1/ship/00652.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-1/ship/14725.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-1/ship/14205.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-1/ship/06477.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-1/ship/02952.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-1/ship/22929.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-1/ship/00357.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-1/ship/16746.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-1/ship/07651.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-1/ship/13713.jpg
Saved: /home/gridsan/manderson/ovdsat

In [7]:
# For test

for M in [1]:#, 2, 3, 4, 5]:
    print(f'================ Processing test M={M} ================')
    input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_test-{M}'
    output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-{M}'
    process_directory(input_root, output_root)
    print()

================ Processing test M=1 ================
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/10737.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/03455.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/19066.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/08641.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/09800.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/08210.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/09074.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/07361.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/14861.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/01843.jpg
Saved: /home/gri

Extract 10 background crops per sample

In [8]:
def generate_background_boxes_from_mask(mask_array, num_b, min_size=30, max_size=200, max_iter=100):
    """
    Generate boxes that do NOT intersect with non-zero mask areas.
    """
    h, w = mask_array.shape
    boxes = []

    for _ in range(num_b):
        valid_box = False
        count = 0
        while not valid_box and count < max_iter:
            count += 1
            width = random.randint(min_size, min(max_size, w))
            height = random.randint(min_size, min(max_size, h))
            x = random.randint(0, w - width)
            y = random.randint(0, h - height)
            x2 = x + width
            y2 = y + height

            # Check if the box intersects with foreground mask
            if np.any(mask_array[y:y2, x:x2] > 0):
                continue  # intersecting: skip
            boxes.append((x, y, x2, y2))
            valid_box = True

    return boxes

def generate_background_crops_from_masks(input_dir, output_root, num_b=10, min_size=30, max_size=200):
    input_path = Path(input_dir)
    background_dir = Path(output_root) / "background"
    background_dir.mkdir(parents=True, exist_ok=True)

    for root, _, files in os.walk(input_path):
        for file in tqdm(files):
            if file.endswith(".jpg") and not file.endswith(".mask.jpg"):
                image_file = Path(root) / file
                mask_file = image_file.with_name(f"{image_file.stem}.mask.jpg")

                if not mask_file.exists():
                    print(f"[!] No mask found for {image_file.name}, skipping.")
                    continue

                try:
                    image = Image.open(image_file).convert("RGB")
                    mask = Image.open(mask_file).convert("L")
                except Exception as e:
                    print(f"[✗] Error loading {image_file.name}: {e}")
                    continue

                mask_array = np.array(mask)
                boxes = generate_background_boxes_from_mask(mask_array, num_b, min_size, max_size)

                for i, (x1, y1, x2, y2) in enumerate(boxes):
                    crop = image.crop((x1, y1, x2, y2))
                    crop_filename = f"{image_file.stem}_background{i}.jpg"
                    crop.save(background_dir / crop_filename)
                    #print(f"[✓] Saved: {background_dir / crop_filename}")

In [52]:
# For train

for N in [5, 10, 30]:
    for M in [1]:
        print(f'================ Processing train N={N}, M={M} ================')
        input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_N{N}-{M}'
        output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/train/dior_N{N}-{M}'
        generate_background_crops_from_masks(input_root, output_root)
        print()

================ Processing train N=5, M=1 ================


0it [00:00, ?it/s]
100%|██████████| 10/10 [00:00<00:00, 12.51it/s]



================ Processing train N=10, M=1 ================


0it [00:00, ?it/s]
100%|██████████| 20/20 [00:01<00:00, 16.24it/s]



================ Processing train N=30, M=1 ================


0it [00:00, ?it/s]
100%|██████████| 20/20 [00:01<00:00, 16.87it/s]

In [53]:
# For val

for M in [1]:
    print(f'================ Processing val M={M} ================')
    input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_val-{M}'
    output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-{M}'
    generate_background_crops_from_masks(input_root, output_root)
    print()

================ Processing val M=1 ================


0it [00:00, ?it/s]
100%|██████████| 20/20 [00:00<00:00, 24.26it/s]

In [9]:
# For test

for M in [1]:#, 2, 3, 4, 5]:
    print(f'================ Processing test M={M} ================')
    input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_test-{M}'
    output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-{M}'
    generate_background_crops_from_masks(input_root, output_root, num_b=1) # just get one crop per image
    print()

================ Processing test M=1 ================


0it [00:00, ?it/s]
100%|██████████| 200/200 [00:01<00:00, 103.67it/s]

## Create images + masks (init_data)

In [4]:
import os
import json
import cv2
import numpy as np
from PIL import Image
from pycocotools.coco import COCO

def coco_to_seg(annotation_file, image_directory, save_path, subset_size=None):
    '''
    Convert COCO annotations to segmentation masks in class directories for prototype initialization.

    Args:
        annotation_file (str): Path to the COCO annotation file.
        image_directory (str): Path to the directory containing the images.
        save_path (str): Path to the directory where the segmentation masks will be saved.
    '''
    # Create the output directory if it doesn't exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Load the COCO annotation file
    coco = COCO(annotation_file)

    # Get the list of category IDs
    category_ids = coco.getCatIds()


    # Loop through each category
    for category_id in category_ids:
        # Get the category information
        category_info = coco.loadCats(category_id)[0]
        category_name = category_info['name']
        category_directory = os.path.join(save_path, category_name)

        # Create a directory for the category
        if not os.path.exists(category_directory):
            os.makedirs(category_directory)

        # Get the image IDs containing the selected category
        image_ids = coco.getImgIds(catIds=category_id)
        
        # Subsample images if subset_size is given
        if subset_size is not None and subset_size < len(image_ids):
            image_ids = random.sample(image_ids, subset_size)

        for image_id in image_ids:
            # Load the image and annotations
            image_info = coco.loadImgs(image_id)[0]
            image_filename = image_info['file_name']
            image = cv2.imread(os.path.join(image_directory, image_filename))
            annotations = coco.loadAnns(coco.getAnnIds(imgIds=image_id, catIds=category_id))

            # Create a mask for the selected category
            mask = np.zeros(image.shape[:2], dtype=np.uint8)
            for annotation in annotations:
                bbox = list(map(int, annotation['bbox']))
                x, y, w, h = bbox
                mask[y:y + h, x:x + w] = 255

            # Save the image and mask
            image_filename_without_ext = os.path.splitext(image_filename)[0]
            mask_filename = f"{image_filename_without_ext}.mask{os.path.splitext(image_filename)[1]}"
            cv2.imwrite(os.path.join(category_directory, image_filename), image)
            cv2.imwrite(os.path.join(category_directory, mask_filename), mask)

        print(f"Processed category: {category_name}")

    print("Processing complete.")

In [45]:
# For train
subset_size = 10
for N in [5, 10, 30]:
    for M in [1]:
        annotation_file=f'/home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N{N}-{M}.json'
        image_directory='/home/gridsan/manderson/ovdsat/data/dior/JPEGImages'
        save_path=f'/home/gridsan/manderson/ovdsat/data/init_data/dior_N{N}-{M}'
        coco_to_seg(annotation_file, image_directory, save_path, subset_size)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Processed category: groundtrackfield
Processed category: baseballfield
Processed category: bridge
Processed category: Expressway-toll-station
Processed category: vehicle
Processed category: airplane
Processed category: airport
Processed category: tenniscourt
Processed category: trainstation
Processed category: storagetank
Processed category: stadium
Processed category: windmill
Processed category: ship
Processed category: golffield
Processed category: overpass
Processed category: chimney
Processed category: dam
Processed category: basketballcourt
Processed category: harbor
Processed category: Expressway-Service-area
Processing complete.
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Processed category: groundtrackfield
Processed category: baseballfield
Processed category: bridge
Processed category: Expressway-toll-station
Processed category: vehicle
Processed category:

In [46]:
# For val
subset_size = 10
for M in [1]:
    annotation_file=f'/home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-{M}.json'
    image_directory='/home/gridsan/manderson/ovdsat/data/dior/JPEGImages'
    save_path=f'/home/gridsan/manderson/ovdsat/data/init_data/dior_val-{M}'
    coco_to_seg(annotation_file, image_directory, save_path, subset_size)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Processed category: groundtrackfield
Processed category: baseballfield
Processed category: bridge
Processed category: Expressway-toll-station
Processed category: vehicle
Processed category: airplane
Processed category: airport
Processed category: tenniscourt
Processed category: trainstation
Processed category: storagetank
Processed category: stadium
Processed category: windmill
Processed category: ship
Processed category: golffield
Processed category: overpass
Processed category: chimney
Processed category: dam
Processed category: basketballcourt
Processed category: harbor
Processed category: Expressway-Service-area
Processing complete.


In [5]:
# For test
subset_size = 100
for M in [1]:#, 2, 3, 4, 5]:
    annotation_file=f'/home/gridsan/manderson/ovdsat/data/dior/val_coco-{M}.json'
    image_directory='/home/gridsan/manderson/ovdsat/data/dior/JPEGImages'
    save_path=f'/home/gridsan/manderson/ovdsat/data/init_data/dior_test-{M}'
    coco_to_seg(annotation_file, image_directory, save_path, subset_size)

loading annotations into memory...
Done (t=0.46s)
creating index...
index created!
Processed category: groundtrackfield
Processed category: baseballfield
Processed category: bridge
Processed category: Expressway-toll-station
Processed category: vehicle
Processed category: airplane
Processed category: airport
Processed category: tenniscourt
Processed category: trainstation
Processed category: storagetank
Processed category: stadium
Processed category: windmill
Processed category: ship
Processed category: golffield
Processed category: overpass
Processed category: chimney
Processed category: dam
Processed category: basketballcourt
Processed category: harbor
Processed category: Expressway-Service-area
Processing complete.


## From annotations file (don't use)
* Do the same but load from a .json file (gives image path)
* Select a subset of N images (or else it will take too long)

In [9]:
import json
import random
from pathlib import Path
from PIL import Image
import numpy as np
import xml.etree.ElementTree as ET

def apply_rectangular_mask_and_crop(image: Image.Image, mask: Image.Image) -> Image.Image:
    """Crops the image to the rectangular mask bounding box only (no masking)."""
    mask_array = np.array(mask.convert("L"))
    ys, xs = np.where(mask_array > 0)

    if xs.size == 0 or ys.size == 0:
        return Image.new("RGB", (1, 1))  # Empty image if no mask

    x_min, x_max = xs.min(), xs.max()
    y_min, y_max = ys.min(), ys.max()
    return image.crop((x_min, y_min, x_max + 1, y_max + 1))

def load_annotations(xml_path: Path):
    """Parse a Pascal VOC-style XML file and return object entries (optional)."""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        objects = []
        for obj in root.findall("object"):
            name = obj.find("name").text
            bbox = obj.find("bndbox")
            box = {
                "name": name,
                "bbox": (
                    int(bbox.find("xmin").text),
                    int(bbox.find("ymin").text),
                    int(bbox.find("xmax").text),
                    int(bbox.find("ymax").text),
                ),
            }
            objects.append(box)
        return objects
    except Exception as e:
        print(f"[!] Failed to parse {xml_path.name}: {e}")
        return []

def process_from_json(
    json_path: str,
    image_root: str,
    annotations_dir: str,
    output_dir: str,
    subset_size: int = 100
):
    image_root = Path(image_root)
    annotations_dir = Path(annotations_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    with open(json_path, "r") as f:
        data = json.load(f)
    
    images = data["images"]
    if subset_size > 0:
        images = random.sample(images, subset_size)

    for img_info in images:
        try:
            file_name = img_info["file_name"]
            folder = img_info.get("folder", "")
            image_path = image_root / folder / file_name
            mask_path = image_path.with_name(image_path.stem + ".mask.jpg")
            xml_path = annotations_dir / (image_path.stem + ".xml")

            if not mask_path.exists():
                print(f"[!] Mask not found: {mask_path}")
                continue

            image = Image.open(image_path).convert("RGB")
            mask = Image.open(mask_path).convert("L")
            masked_cropped = apply_rectangular_mask_and_crop(image, mask)

            # Optional: load annotations
            _ = load_annotations(xml_path)

            # Save with preserved folder structure
            output_file = output_path / folder / file_name
            output_file.parent.mkdir(parents=True, exist_ok=True)
            masked_cropped.save(output_file)
            print(f"[✓] Saved: {output_file}")

        except Exception as e:
            print(f"[✗] Error processing {img_info}: {e}")


In [10]:
M=1

process_from_json(
    json_path=f"/home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-{M}.json",
    image_root="/home/gridsan/manderson/ovdsat/data/dior/JPEGImages",
    annotations_dir="/home/gridsan/manderson/ovdsat/data/dior/Annotations",
    output_dir=f"/home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior-{M}",
    subset_size=10
)

[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/00499.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/01711.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/21022.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/17339.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/21084.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/19542.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/12238.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/02348.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/03136.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/17443.mask.jpg
